In [ ]:
# --- Distribution audit using already-loaded DataFrames: train, test, orig ---
import pandas as pd
import numpy as np

# If you DIDN'T pre-load them, uncomment these:
train = pd.read_csv('/kaggle/input/playground-series-s5e10/train.csv')
test  = pd.read_csv('/kaggle/input/playground-series-s5e10/test.csv')
orig  = pd.concat([
    pd.read_csv('/kaggle/input/simulated-roads-accident-data/synthetic_road_accidents_100k.csv'),
    pd.read_csv('/kaggle/input/simulated-roads-accident-data/synthetic_road_accidents_10k.csv'),
    pd.read_csv('/kaggle/input/simulated-roads-accident-data/synthetic_road_accidents_2k.csv'),
], ignore_index=True)

CATS = ["road_type","time_of_day","weather","lighting","holiday","school_season"]
HAS_TARGET = "accident_risk" in train.columns

def summarize_feature(df: pd.DataFrame, col: str, with_target: bool) -> pd.DataFrame:
    if with_target and "accident_risk" in df.columns:
        # FIX: SeriesGroupBy named aggregation uses strings, not (col, func) tuples
        gb = df.groupby(col)["accident_risk"].agg(rows="count", mean="mean", std="std")
        out = gb.reset_index().sort_values("rows", ascending=False)
    else:
        out = (df.groupby(col)
                 .size()
                 .reset_index(name="rows")
                 .sort_values("rows", ascending=False))
        out["mean"] = np.nan
        out["std"]  = np.nan
    out["share"] = (out["rows"] / len(df)).round(4)
    return out[[col, "rows", "share", "mean", "std"]]

def binned_accidents(df: pd.DataFrame, q: int = 10):
    if "num_reported_accidents" not in df.columns:
        return None
    return pd.qcut(df["num_reported_accidents"], q=q, duplicates="drop")

print("=== Shapes ===")
print(f"train: {train.shape} | test: {test.shape} | orig: {orig.shape}\n")

# Overall target stats (train only)
if HAS_TARGET:
    print("=== Train target stats ===")
    print(train["accident_risk"].agg(["count","mean","std","min","max"]).to_string(), "\n")

results = {}

# Per-column summaries
for col in CATS:
    print(f"=== {col} (train) ===")
    tr = summarize_feature(train, col, with_target=True)
    display(tr.head(20))
    results[f"train_{col}"] = tr

    print(f"=== {col} (test) ===")
    te = summarize_feature(test, col, with_target=False)
    display(te.head(20))
    results[f"test_{col}"] = te

    print(f"=== {col} (orig) ===")
    og = summarize_feature(orig, col, with_target=("accident_risk" in orig.columns))
    display(og.head(20))
    results[f"orig_{col}"] = og
    print()

# Binned accident history
train_bins = binned_accidents(train, q=10)
if train_bins is not None:
    dfb = train.assign(num_acc_bin=train_bins)
    print("=== num_reported_accidents (binned) — train ===")
    summ = (dfb.groupby("num_acc_bin")["accident_risk"]
              .agg(rows="count", mean="mean", std="std")
              .reset_index()
              .sort_values("rows", ascending=False))
    summ["share"] = (summ["rows"]/len(train)).round(4)
    display(summ)
    results["train_num_acc_bins"] = summ
    print()

# Composite key coverage to spot rare combos (great for Group/StratifiedGroupKFold)
def composite_key(df: pd.DataFrame) -> pd.Series:
    cols = ["road_type","time_of_day","weather"]  # adjust if needed
    return df[cols].astype(str).agg("|".join, axis=1)

print("=== Composite key coverage (road_type|time_of_day|weather) ===")
train["ckey"] = composite_key(train)
test["ckey"]  = composite_key(test)
orig["ckey"]  = composite_key(orig)

train_ck = (train["ckey"].value_counts() / len(train)).rename("share_train")
test_ck  = (test["ckey"].value_counts()  / len(test)).rename("share_test")
orig_ck  = (orig["ckey"].value_counts()  / len(orig)).rename("share_orig")

ck = pd.concat([train_ck, test_ck, orig_ck], axis=1).fillna(0).sort_values("share_test", ascending=False)
ck["abs_gap_train_vs_test"] = (ck["share_train"] - ck["share_test"]).abs().round(4)
ck["abs_gap_orig_vs_test"]  = (ck["share_orig"]  - ck["share_test"]).abs().round(4)
display(ck.head(30))

# Flag rare combos in train that might cause fold sparsity
rare_threshold = 0.002  # 0.2% of rows — tweak as needed
rare_train = ck[ck["share_train"] < rare_threshold].sort_values("share_train")
print(f"Rare combos in train (<{rare_threshold*100:.1f}% of rows): {len(rare_train)}")
display(rare_train.head(30))

print("\n--- Use this to pick stratification/group keys for your new CV splits. ---")


=== Shapes ===
train: (517754, 14) | test: (172585, 13) | orig: (112000, 13)

=== Train target stats ===
count    517754.000000
mean          0.352377
std           0.166417
min           0.000000
max           1.000000 

=== road_type (train) ===


,road_type,rows,share,mean,std
0,highway,173672,0.3354,0.349734,0.165922
1,rural,172719,0.3336,0.349997,0.167185
2,urban,171363,0.3310,0.357456,0.166027


=== road_type (test) ===


,road_type,rows,share,mean,std
0,highway,58080,0.3365,NaN,NaN
1,rural,57409,0.3326,NaN,NaN
2,urban,57096,0.3308,NaN,NaN


=== road_type (orig) ===


...


=== Composite key coverage (road_type|time_of_day|weather) ===


,share_train,share_test,share_orig,abs_gap_train_vs_test,abs_gap_orig_vs_test
ckey,,,,,
urban|evening|foggy,0.040963,0.041267,0.036946,0.0003,0.0043
highway|morning|clear,0.040228,0.040774,0.038080,0.0005,0.0027
highway|evening|foggy,0.040092,0.039951,0.037607,0.0001,0.0023
rural|evening|foggy,0.040280,0.039893,0.036134,0.0004,0.0038
highway|afternoon|clear,0.039691,0.039789,0.036973,0.0001,0.0028
rural|afternoon|clear,0.039109,0.039459,0.037848,0.0003,0.0016
rural|morning|clear,0.039322,0.039395,0.036964,0.0001,0.0024
highway|afternoon|foggy,0.038360,0.039239,0.036991,0.0009,0.0022
highway|evening|clear,0.038694,0.038833,0.037884,0.0001,0.0009


Rare combos in train (<0.2% of rows): 0


,share_train,share_test,share_orig,abs_gap_train_vs_test,abs_gap_orig_vs_test
ckey,,,,,



--- Use this to pick stratification/group keys for your new CV splits. ---
